# Data Locality

## Load Reference Data

In [1]:
import pandas as pd

### Load State & Region Data

In [2]:
df = pd.read_csv('data/imputed/state-predictions.csv', encoding='utf-8')
df = df.filter(items=['state', 'region', 'latitude', 'longitude']).drop_duplicates()
df

,state,region,latitude,longitude
0,Connecticut,Northeast,41.6219,-72.7273
1,Delaware,Northeast,38.9896,-75.5050
2,Maine,Northeast,45.3695,-69.2428
3,Maryland,Northeast,39.0550,-76.7909
4,Massachusetts,Northeast,42.2596,-71.8083
5,New Hampshire,Northeast,43.6805,-71.5811
6,New Jersey,Northeast,40.1907,-74.6728
7,New York,Northeast,42.9538,-75.5268
8,Pennsylvania,Northeast,40.8781,-77.7996
9,Rhode Island,Northeast,41.6762,-71.5562


### Connect to Redis

In [3]:
import redis
conn = redis.Redis()

### Load State Data

In [4]:
for index, row in df.iterrows():
    state = row['state']
    region = row['region']
    latitude = row['latitude']
    longitude = row['longitude']
    conn.geoadd('woodchuckin:states:coords', longitude, latitude, state)
    conn.set(f'woodchuckin:states:{state}:region', region)
    conn.sadd(f'woodchuckin:regions:{region}:state', state)

### Load Regional Forecasts

In [5]:
df_region = pd.read_csv('data/forecast/forecasted-monthly-temperatures-by-region.csv', encoding='utf-8')
df_region

,region,date,temperature
0,Northeast,202102,28.8
1,Northeast,202103,38.3
2,Upper Midwest,202102,19.7
3,Upper Midwest,202103,34.5
4,Ohio Valley,202102,35.9
5,Ohio Valley,202103,48.0


In [6]:
for index, row in df_region.iterrows():
    region = row['region']
    date = row['date']
    temp = row['temperature']
    conn.set(f'woodchuckin:forecast:{date}:region:{region}', temp)

### Load State Forecasts

In [7]:
df_state = pd.read_csv('data/forecast/forecasted-monthly-temperatures-by-state.csv', encoding='utf-8')
df_state

,state,date,temperature
0,Connecticut,202102,34.0
1,Connecticut,202103,41.6
2,Delaware,202102,41.9
3,Delaware,202103,49.5
4,Illinois,202102,31.9
5,Illinois,202103,44.4
6,Indiana,202102,32.8
7,Indiana,202103,44.6
8,Iowa,202102,24.8
9,Iowa,202103,40.3


In [8]:
for index, row in df_state.iterrows():
    state = row['state']
    date = row['date']
    temp = row['temperature']
    conn.set(f'woodchuckin:forecast:{date}:state:{state}', temp)

## Setup Scripting

### Register the Trigger

Delete existing triggers.

In [138]:
registrations = conn.execute_command('RG.DUMPREGISTRATIONS')
for reg in registrations:
  id = reg[1]
  conn.execute_command('RG.UNREGISTER', id)

Add our trigger.

In [139]:
conn.execute_command('RG.PYEXECUTE', """

def to_dictionary(data):
  return { 'latitude': data[1], 'longitude': data[2] }
  
def blend_reference_data(data):
  latitude, longitude = data['latitude'], data['longitude']
  
  state = execute('GEORADIUS', 'woodchuckin:states:coords', longitude, latitude, 10000, 'mi', 'ASC')[0]
  region = execute('GET', f'woodchuckin:states:{state}:region')

  data['feb_state_temp'] = execute('GET', f'woodchuckin:forecast:202102:state:{state}')
  data['mar_state_temp'] = execute('GET', f'woodchuckin:forecast:202103:state:{state}')
  data['feb_region_temp'] = execute('GET', f'woodchuckin:forecast:202102:region:{region}')
  data['mar_region_temp'] = execute('GET', f'woodchuckin:forecast:202103:region:{region}')
  
  return data
  
def predict(data):
  execute('AI.TENSORSET', 'woodchuckin:in', 'FLOAT', 1, 6, 'VALUES',
    data['latitude'], data['longitude'],
    data['feb_state_temp'], data['mar_state_temp'],
    data['feb_region_temp'], data['mar_region_temp'])
    
  execute('AI.MODELRUN', 'woodchuckin', 'INPUTS', 'woodchuckin:in',
          'OUTPUTS', 'woodchuckin:out', 'woodchuckin:scores')

  return execute('AI.TENSORGET', 'woodchuckin:out', 'VALUES')[0]
  
def decode(data):
  return 'Full Shadow' if (data == 1) else 'No Shadow'

gb = GearsBuilder('CommandReader')
gb.map(to_dictionary)
gb.map(blend_reference_data)
gb.map(predict)
gb.map(decode)
gb.register(trigger='woodchuckin')

""")

b'OK'

### Invoke Function

In [140]:
latitude, longitude = 44.6243, -89.9941
conn.execute_command('RG.TRIGGER', 'woodchuckin', latitude, longitude)

[b'Full Shadow']